In [1]:
!pip install autogluon

Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of huggingface-hub[torch] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple ve

In [3]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor


In [5]:
num_samples = 500
data_list = []

for i in range(num_samples):
    is_ko = np.random.rand() > 0.7  

    if is_ko:
        status = "KO"
        feature_A = np.random.normal(loc=100, scale=10) 
        feature_B = np.random.normal(loc=50, scale=2)   
        feature_C = np.random.normal(loc=18, scale=5) 
    else:
        status = "OK"
        feature_A = np.random.normal(loc=20, scale=15) 
        feature_B = np.random.normal(loc=50, scale=20)
        feature_C = np.random.normal(loc=25, scale=5)

    feature_D = np.random.normal(loc=0, scale=1)

    data_list.append([feature_A, feature_B, feature_C, feature_D, status])

columns = ['feature_A', 'feature_B', 'feature_C', 'feature_D', 'status']
train_data = pd.DataFrame(data_list, columns=columns)

train_data.head()

,feature_A,feature_B,feature_C,feature_D,status
0,106.389851,48.685434,24.514438,1.501245,KO
1,28.840387,55.927185,24.047195,-1.712662,OK
2,32.574957,99.102474,24.639858,2.881571,OK
3,12.560110,81.087068,20.630862,-0.503385,OK
4,130.372749,51.507307,19.640574,-0.426312,KO


In [6]:
label_column = 'status'

predictor = TabularPredictor(label=label_column, path='autogluon_models')

predictor.fit(train_data, time_limit=60)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #35~22.04.1-Ubuntu SMP Mon May 26 18:08:30 UTC 2025
CPU Count:          4
Memory Avail:       11.31 GB / 15.62 GB (72.4%)
Disk Space Avail:   15.79 GB / 31.33 GB (50.4%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU memory.
	presets='best'    : Maximize accuracy. Recommended for most users. Use in c

In [7]:
feature_importance = predictor.feature_importance(data=train_data)

print("Feature Importance:")
print(feature_importance)

Computing feature importance via permutation shuffling for 4 features using 500 rows with 5 shuffle sets...
	0.38s	= Expected runtime (0.08s per shuffle set)
	0.15s	= Actual runtime (Completed 5 of 5 shuffle sets)


Feature Importance:
           importance    stddev       p_value  n  p99_high   p99_low
feature_A      0.4396  0.012522  7.891743e-08  5  0.465383  0.413817
feature_C      0.0024  0.002191  3.524200e-02  5  0.006911 -0.002111
feature_D      0.0012  0.001095  3.524200e-02  5  0.003456 -0.001056
feature_B      0.0004  0.000894  1.869505e-01  5  0.002242 -0.001442


In [8]:
predictor.leaderboard(train_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMLarge,1.000,1.00,accuracy,0.000998,0.000360,0.210803,0.000998,0.000360,0.210803,1,True,11
1,CatBoost,1.000,1.00,accuracy,0.002412,0.000507,0.627209,0.002412,0.000507,0.627209,1,True,5
2,NeuralNetFastAI,1.000,1.00,accuracy,0.012694,0.009591,0.955935,0.012694,0.009591,0.955935,1,True,8
3,WeightedEnsemble_L2,1.000,1.00,accuracy,0.013902,0.010241,1.021289,0.001208,0.000650,0.065355,2,True,12
4,ExtraTreesGini,1.000,1.00,accuracy,0.072316,0.034103,0.403718,0.072316,0.034103,0.403718,1,True,6
5,RandomForestGini,1.000,1.00,accuracy,0.073805,0.043880,0.585268,0.073805,0.043880,0.585268,1,True,3
6,RandomForestEntr,1.000,1.00,accuracy,0.075223,0.033472,0.455655,0.075223,0.033472,0.455655,1,True,4
7,ExtraTreesEntr,1.000,1.00,accuracy,0.089617,0.033905,0.412288,0.089617,0.033905,0.412288,1,True,7
8,LightGBM,0.998,1.00,accuracy,0.000766,0.000375,0.146943,0.000766,0.000375,0.146943,1,True,2
9,NeuralNetTorch,0.998,0.99,accuracy,0.005810,0.002770,2.153192,0.005810,0.002770,2.153192,1,True,10


In [11]:
import pandas as pd
from autogluon.tabular import TabularPredictor

# Step 1: Load the data
file_path = "../data/raw/train.csv"
data = pd.read_csv(file_path)

# Step 2: Train the model
label_column = 'Survived'
predictor = TabularPredictor(label=label_column, path='autogluon_models')
predictor.fit(data, time_limit=60)

# Step 3: Get feature importance
feature_importance = predictor.feature_importance(data=data)
print("Feature Importance:")
print(feature_importance)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #35~22.04.1-Ubuntu SMP Mon May 26 18:08:30 UTC 2025
CPU Count:          4
Memory Avail:       10.22 GB / 15.62 GB (65.4%)
Disk Space Avail:   15.65 GB / 31.33 GB (49.9%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU memory.
	presets='best'    : Maximize accuracy. Recommended for most users. Use in c

Feature Importance:
             importance    stddev   p_value  n  p99_high   p99_low
Sex            0.115152  0.006715  0.000001  5  0.128978  0.101325
Name           0.081481  0.012130  0.000057  5  0.106456  0.056507
Ticket         0.062626  0.006467  0.000013  5  0.075942  0.049311
Pclass         0.046914  0.006319  0.000039  5  0.059925  0.033903
Age            0.034119  0.005240  0.000065  5  0.044909  0.023329
SibSp          0.028507  0.002817  0.000011  5  0.034308  0.022707
Parch          0.023120  0.002327  0.000012  5  0.027912  0.018328
Cabin          0.022447  0.006096  0.000593  5  0.034998  0.009895
Fare           0.014590  0.001775  0.000026  5  0.018244  0.010936
Embarked       0.013692  0.001665  0.000026  5  0.017120  0.010265
PassengerId    0.013244  0.002680  0.000191  5  0.018761  0.007726


In [14]:
predictor.leaderboard(data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,ExtraTreesGini,0.962963,0.815642,accuracy,0.067530,0.055514,0.619512,0.067530,0.055514,0.619512,1,True,6
1,RandomForestGini,0.962963,0.815642,accuracy,0.078247,0.057523,0.522995,0.078247,0.057523,0.522995,1,True,3
2,RandomForestEntr,0.962963,0.815642,accuracy,0.080002,0.055830,0.548988,0.080002,0.055830,0.548988,1,True,4
3,ExtraTreesEntr,0.961841,0.810056,accuracy,0.071357,0.056203,0.519780,0.071357,0.056203,0.519780,1,True,7
4,LightGBMLarge,0.928171,0.815642,accuracy,0.003974,0.002892,0.590089,0.003974,0.002892,0.590089,1,True,11
5,LightGBM,0.905724,0.821229,accuracy,0.007267,0.002871,0.307998,0.007267,0.002871,0.307998,1,True,2
6,WeightedEnsemble_L2,0.903479,0.871508,accuracy,0.020092,0.014119,3.056133,0.001280,0.000669,0.061923,2,True,12
7,NeuralNetTorch,0.896745,0.849162,accuracy,0.013553,0.009530,2.295783,0.013553,0.009530,2.295783,1,True,10
8,NeuralNetFastAI,0.892256,0.826816,accuracy,0.019481,0.007278,0.569583,0.019481,0.007278,0.569583,1,True,8
9,XGBoost,0.879910,0.815642,accuracy,0.008618,0.004374,0.262110,0.008618,0.004374,0.262110,1,True,9


In [15]:
# Save feature importance to a CSV file
feature_importance.to_csv("../data/processed/feature_importance.csv", index=True)